<a href="https://colab.research.google.com/github/pennya6/data_analysis/blob/master/%EB%B9%85%EB%8D%B0%EC%9D%B4%ED%84%B0%EB%B6%84%EC%84%9D/CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

MNIST 데이터 CNN

In [2]:
import tensorflow_datasets as tfds
import numpy as np
import pandas as pd
import tensorflow as tf

In [3]:
mnist_builder=tfds.builder('mnist')
mnist_builder.download_and_prepare()
datasets=mnist_builder.as_dataset(shuffle_files=False)

local data directory. If you'd instead prefer to read directly from our public
GCS bucket (recommended if you're running on GCP), you can instead pass
`try_gcs=True` to `tfds.load` or set `data_dir=gs://tfds-data/datasets`.



Dl Completed...:   0%|          | 0/4 [00:00<?, ? file/s]


Dataset mnist downloaded and prepared to /root/tensorflow_datasets/mnist/3.0.1. Subsequent calls will reuse this data.


In [4]:
datasets

{'test': <PrefetchDataset shapes: {image: (28, 28, 1), label: ()}, types: {image: tf.uint8, label: tf.int64}>,
 'train': <PrefetchDataset shapes: {image: (28, 28, 1), label: ()}, types: {image: tf.uint8, label: tf.int64}>}

In [6]:
#딕셔너리에 있는 밸류값 뽑아서 만들어주기
# test key의 value를 뽑아서 test 데이터로 변경
mnist_train_orig=datasets['train']
mnist_test_orig=datasets['test']

In [7]:
mnist_test_orig

<PrefetchDataset shapes: {image: (28, 28, 1), label: ()}, types: {image: tf.uint8, label: tf.int64}>

In [8]:
#데이터셋 표준화
# 텐서플로우는 타입에 예민하기 때문에 cast이후 나눠주기
mnist_train=mnist_train_orig.map(lambda item : (tf.cast(item['image'],tf.float32)/255.0,tf.cast(item['label'],tf.int32)))
mnist_test=mnist_test_orig.map(lambda item : (tf.cast(item['image'],tf.float32)/255.0,tf.cast(item['label'],tf.int32)))

In [9]:
BUFFER_SIZE=10000
BATCH_SIZE=64
tf.random.set_seed(1)
mnist_train=mnist_train.shuffle(buffer_size=BUFFER_SIZE,reshuffle_each_iteration=False)
mnist_valid=mnist_train.take(10000).batch(BATCH_SIZE)
mnist_train=mnist_train.skip(10000).batch(BATCH_SIZE)

In [10]:
cnn=tf.keras.Sequential()
#padding='same'은 28*28 비율을 입력층과 동일하게 하겠다. strides는 1, 필터는 32개, 필터의 크기는 5*5
cnn.add(tf.keras.layers.Conv2D(filters=32,kernel_size=(5,5),strides=(1,1),padding='same',data_format='channels_last',name='conv_1',activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=(2,2),name='pool_1'))
cnn.add(tf.keras.layers.Conv2D(filters=64,kernel_size=(5,5),strides=(1,1),padding='same',name='conv_2',activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=(2,2),name='pool_2'))

cnn.add(tf.keras.layers.Flatten()) #MLP처럼 펼쳐주는 작업, 입력층과 비슷한 느낌
cnn.add(tf.keras.layers.Dense(units=1024,name='fc_1',activation='relu')) #은닉층 유닛수는 1024
cnn.add(tf.keras.layers.Dropout(rate=0.5)) #드롭아웃 0.5의 확률로 랜덤하게 사용함
cnn.add(tf.keras.layers.Dense(units=10,name='fc_2',activation='softmax')) #출력층 다진분류이기 때문에 softmax사용

In [14]:
cnn.build(input_shape=(None,28,28,1))

In [15]:
cnn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv_1 (Conv2D)             (None, 28, 28, 32)        832       
                                                                 
 pool_1 (MaxPooling2D)       (None, 14, 14, 32)        0         
                                                                 
 conv_2 (Conv2D)             (None, 14, 14, 64)        51264     
                                                                 
 pool_2 (MaxPooling2D)       (None, 7, 7, 64)          0         
                                                                 
 flatten (Flatten)           (None, 3136)              0         
                                                                 
 fc_1 (Dense)                (None, 1024)              3212288   
                                                                 
 dropout (Dropout)           (None, 1024)              0

In [18]:
cnn.compile(optimizer=tf.keras.optimizers.Adam(),
            loss=tf.keras.losses.SparseCategoricalCrossentropy(),
            metrics=['accuracy'])
NUM_EPOCHS=20
history=cnn.fit(mnist_train,epochs=NUM_EPOCHS,
                validation_data=mnist_valid,shuffle=True)

Epoch 1/20
782/782 [==============================] - 160s 201ms/step - loss: 0.1395 - accuracy: 0.9564 - val_loss: 0.0484 - val_accuracy: 0.9846
Epoch 2/20
782/782 [==============================] - 153s 195ms/step - loss: 0.0456 - accuracy: 0.9860 - val_loss: 0.0427 - val_accuracy: 0.9873
Epoch 3/20
782/782 [==============================] - 152s 194ms/step - loss: 0.0304 - accuracy: 0.9903 - val_loss: 0.0351 - val_accuracy: 0.9889
Epoch 4/20
782/782 [==============================] - 153s 195ms/step - loss: 0.0220 - accuracy: 0.9933 - val_loss: 0.0402 - val_accuracy: 0.9887
Epoch 5/20
782/782 [==============================] - 153s 195ms/step - loss: 0.0172 - accuracy: 0.9946 - val_loss: 0.0442 - val_accuracy: 0.9885
Epoch 6/20
782/782 [==============================] - 153s 194ms/step - loss: 0.0169 - accuracy: 0.9949 - val_loss: 0.0404 - val_accuracy: 0.9893
Epoch 7/20
782/782 [==============================] - 153s 194ms/step - loss: 0.0129 - accuracy: 0.9960 - val_loss: 0.0387 -

In [19]:
cnn.evaluate(mnist_test.batch(20))

500/500 [==============================] - 11s 21ms/step - loss: 0.0429 - accuracy: 0.9928


[0.042916372418403625, 0.9927999973297119]